In [1]:
%load_ext autoreload
%autoreload 2

%cd ../

/Users/hoangle/Uni/Thesis


In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
import pandas as pd
import torch
from recbole.config import Config
from recbole.data import create_dataset
from recbole.utils import (
    get_model,
    get_trainer,
    init_seed,
    ModelType
)

import src.utils as utils
from src.real_temporal import SimulatedOnlineSequentialDataset, SimulatedOnlineDataset

# Load checkpoint

In [10]:
separate_activeness = False
cutoff_time = 976324045

In [11]:
path = "/Users/hoangle/Desktop/GRU4Rec-Aug-05-2024_16-36-56.pth"

In [12]:
# config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_file=path)
chkpt = torch.load(path, map_location=torch.device('cpu'))

config = chkpt['config']
config['device'] = torch.device('cpu')
config['use_gpu'] = False

if separate_activeness is True:
    if config['cutoff_time'] is None:
        config['cutoff_time'] = cutoff_time

init_seed(config["seed"], config["reproducibility"])

# Define dataset

In [13]:
if config["scheme"] == "so":
    match (config["MODEL_TYPE"]):
        case ModelType.GENERAL | ModelType.CONTEXT | ModelType.TRADITIONAL:
            ds = "SimulatedOnlineDataset"
        case ModelType.SEQUENTIAL:
            ds = "SimulatedOnlineSequentialDataset"
        case _:
            print(f"model type: {config['MODEL_TYPE']}")
            raise NotImplementedError()

    dataset = eval(ds)(config)
elif config["scheme"] == "loo":
    dataset = create_dataset(config)
else:
    raise NotImplementedError()

separate_activeness = config["scheme"] == "loo"
loaders = utils.get_loader(
    dataset, config, separate_activeness, config["cutoff_time"]
)

# Define model
model_name = config["model"]
model = get_model(model_name)(config, loaders["train"]._dataset).to(config["device"])
model.load_state_dict(chkpt['state_dict'])
model.load_other_parameter(chkpt['other_parameter'])

# Define trainer
trainer = get_trainer(config["MODEL_TYPE"], config["model"])(config, model)

# Start evaluation

In [ ]:
result_val_ns = dict(trainer.evaluate(loaders["val_ns"], model_file=path))
result_val_non = dict(trainer.evaluate(loaders["val_non"], model_file=path))

result_test_ns = dict(trainer.evaluate(loaders["test_ns"], model_file=path))
result_test_non = dict(trainer.evaluate(loaders["test_non"], model_file=path))

if separate_activeness is True:
    result_val_act_ns = dict(trainer.evaluate(loaders["val_act_ns"], model_file=path))
    result_test_act_ns = dict(trainer.evaluate(loaders["test_act_ns"], model_file=path))
    result_val_inact_ns = dict(trainer.evaluate(loaders["val_inact_ns"], model_file=path))
    result_test_inact_ns = dict(trainer.evaluate(loaders["test_inact_ns"], model_file=path))

    result_val_act_non = dict(trainer.evaluate(loaders["val_act_non"], model_file=path))
    result_test_act_non = dict(trainer.evaluate(loaders["test_act_non"], model_file=path))
    result_val_inact_non = dict(trainer.evaluate(loaders["val_inact_non"], model_file=path))
    result_test_inact_non = dict(trainer.evaluate(loaders["test_inact_non"], model_file=path))

In [14]:
results = {}

if separate_activeness is True:
    pairs = [
        ("Inactive", test_data_inactive),
        ("Active", test_data_active),
    ]

    for tag, test_data in pairs:
        test_result = trainer.evaluate(test_data, model_file=path)

        results[tag] = dict(test_result)

else:
    test_result = trainer.evaluate(test_data, model_file=path)

    results['test_all'] = dict(test_result)

pd.DataFrame(results).transpose()